In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# warnings.simplefilter('ignore')
import time
import lightgbm as lgb
from lightgbm import early_stopping,log_evaluation
# pip 安装3.3.2版本 lightgbm，conda无法安装
from keras.layers import MaxPooling1D,Conv1D,UpSampling1D
from keras.layers import Dense, Input
from keras.models import Model
from keras import initializers
from sklearn.metrics import f1_score

In [2]:
# 读取
data_to_split ='F:\\zzqaq\\data\\data_to_split.csv'
data_to_split = pd.read_csv(data_to_split,converters={'code':str,'year':str})
data_to_split[['board','year']] = data_to_split[['code','year']].astype(int)
# 按行业列行分组
grouped = data_to_split.groupby("industry_code")

# 空df
data  = pd.DataFrame()

# 对于每个行业，按1:1的比例随机选择"0"和"1"标签的数据
for name, group in grouped:
    group1  = group[group["label"] == 1]
    num_to_extract_0 = len(group1)
    group0 = group[group['label'] == 0].sample(n=num_to_extract_0, random_state=42)
    data = pd.concat([data, group1, group0])
    
data = data.sample(frac=1, random_state=42)

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

print(train_df.shape)
print(test_df.shape)
train_df.head(2)

(382, 779)
(96, 779)


,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_766,feature_767,label,industry_code,word_count,sentence_count,readability,pos_count,neg_count,tone
7014,603703,2020,603703,0.139735,-0.177732,0.284961,0.033765,-0.164080,-0.896397,0.213114,...,0.305694,0.177161,0,7,4723,146,32.349315,527,129,0.606707
1825,300158,2019,300158,-0.560366,0.024280,0.735965,0.123290,-0.692182,-1.308341,0.413451,...,-1.149953,-0.731047,0,7,8829,249,35.457831,1002,183,0.691139


In [3]:
# 纯文本向量
feature_text = [i for i in data_to_split.columns if
                 i not in ['code', 'label','year','board',
                           'industry_code','word_count','sentence_count','readability',
                           'pos_count','neg_count','tone']]
data_np = data[feature_text]
data_np.shape

(478, 768)

In [4]:
# AE神经网络降维
# 经比较选择将文本向量从768降至128维
def bulid_BPNNmodel(data_np):
#   三维转二维数组后拆分。 
#     batch_size = 512
    batch_size = 478
    num_batches = data_np.shape[0] // batch_size
    data_batches = np.array_split(data_np[:num_batches*batch_size], num_batches)

    # 定义自编码器模型
    input_dim = 768; encoding_dim = 128  # 降维后的维度
    input_layer = Input(shape=(input_dim,))
    print(input_layer.shape)
    
    # 编码层
    h1, h2, h3 = 64, 32, 16  #隐藏层1 2 3
    encoded1 = Dense(h1, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(input_layer)
    encoded2 = Dense(h2, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(encoded1)
    encoded3 = Dense(h3, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(encoded2)
    encoder_output = Dense(encoding_dim)(encoded3)
    # 解码层
    decoded1 = Dense(h3, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(encoder_output)
    decoded2 = Dense(h2, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(decoded1)
    decoded3 = Dense(h1, activation='relu', kernel_initializer=initializers.random_normal(stddev=0.01),bias_initializer='zeros')(decoded2)
    decoded_output = Dense(input_dim, activation='relu',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(decoded3)
    autoencoder = Model(inputs=input_layer, outputs = decoded_output)
    # 编译AE神经网络
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    bs = 128; epochs = max(int(bs / 2), 320)
 
    encoded_outputs = []
    reconstructed_outputs= []
    for i, batch in enumerate(data_batches):
        print(f"Processing batch {i+1}/{num_batches}")  
        print(batch.shape)  # (512, 768)
        # fit()方法默认情况下在每个epoch结束时更新模型的权重参数
        history = autoencoder.fit(batch, batch, epochs=epochs, 
                            validation_split=0.2)
         # 编码数据并保存到文件
        encoder = Model(inputs=input_layer, outputs=encoder_output)
        encoded_data_batch= encoder.predict(batch)
        encoded_outputs.append(encoded_data_batch)
        with open('encoded_np.txt', 'a') as f:
            np.savetxt(f, encoded_data_batch)  # 将编码数据写入文件中
        reconstructed_data_batches = autoencoder.predict(batch)
        reconstructed_outputs.append(reconstructed_data_batches)
    encoded_data = np.concatenate(encoded_outputs, axis=0)
    # 计算重构误差
    reconstructed_data = np.concatenate(reconstructed_outputs, axis=0)
    reconstruction_error = np.mean(np.square(data_np - reconstructed_data))
    print("Reconstruction error:", reconstruction_error)
    return encoded_data,reconstruction_error
encoder_np, reconstruction_np = bulid_BPNNmodel(data_np)
encoder_np = pd.DataFrame(encoder_np)


(None, 768)
Processing batch 1/1
(478, 768)
Epoch 1/320
12/12 [==============================] - 1s 18ms/step - loss: 0.3574 - mse: 0.3574 - val_loss: 0.3594 - val_mse: 0.3594
Epoch 2/320
12/12 [==============================] - 0s 6ms/step - loss: 0.3533 - mse: 0.3533 - val_loss: 0.3541 - val_mse: 0.3541
Epoch 3/320
12/12 [==============================] - 0s 7ms/step - loss: 0.3465 - mse: 0.3465 - val_loss: 0.3450 - val_mse: 0.3450
Epoch 4/320
12/12 [==============================] - 0s 7ms/step - loss: 0.3344 - mse: 0.3344 - val_loss: 0.3265 - val_mse: 0.3265
Epoch 5/320
12/12 [==============================] - 0s 8ms/step - loss: 0.3068 - mse: 0.3068 - val_loss: 0.2939 - val_mse: 0.2939
Epoch 6/320
12/12 [==============================] - 0s 7ms/step - loss: 0.2856 - mse: 0.2856 - val_loss: 0.2781 - val_mse: 0.2781
Epoch 7/320
12/12 [==============================] - 0s 5ms/step - loss: 0.2687 - mse: 0.2687 - val_loss: 0.2591 - val_mse: 0.2591
Epoch 8/320
12/12 [===================

Epoch 63/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 64/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 65/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 66/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1987 - mse: 0.1987 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 67/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 68/320
12/12 [==============================] - 0s 7ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 69/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1987 - mse: 0.1987 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 70/320
12/12 [==============================] - 0s 5ms/step - loss: 0.

Epoch 125/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1985 - mse: 0.1985 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 126/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1986 - mse: 0.1986 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 127/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1986 - mse: 0.1986 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 128/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1985 - mse: 0.1985 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 129/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1986 - mse: 0.1986 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 130/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1986 - mse: 0.1986 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 131/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1985 - mse: 0.1985 - val_loss: 0.1992 - val_mse: 0.1992
Epoch 132/320
12/12 [==============================] - 0s 6ms/step - 

12/12 [==============================] - 0s 6ms/step - loss: 0.1971 - mse: 0.1971 - val_loss: 0.1988 - val_mse: 0.1988
Epoch 187/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1969 - mse: 0.1969 - val_loss: 0.1982 - val_mse: 0.1982
Epoch 188/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1968 - mse: 0.1968 - val_loss: 0.1980 - val_mse: 0.1980
Epoch 189/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1965 - mse: 0.1965 - val_loss: 0.1978 - val_mse: 0.1978
Epoch 190/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1962 - mse: 0.1962 - val_loss: 0.1978 - val_mse: 0.1978
Epoch 191/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1959 - mse: 0.1959 - val_loss: 0.1968 - val_mse: 0.1968
Epoch 192/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1954 - mse: 0.1954 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 193/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1953 -

Epoch 248/320
12/12 [==============================] - 0s 8ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 249/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 250/320
12/12 [==============================] - 0s 7ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 251/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 252/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1937 - val_mse: 0.1937
Epoch 253/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 254/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 255/320
12/12 [==============================] - 0s 6ms/step - 

12/12 [==============================] - 0s 6ms/step - loss: 0.1915 - mse: 0.1915 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 310/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1916 - mse: 0.1916 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 311/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1915 - mse: 0.1915 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 312/320
12/12 [==============================] - 0s 5ms/step - loss: 0.1915 - mse: 0.1915 - val_loss: 0.1927 - val_mse: 0.1927
Epoch 313/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1915 - mse: 0.1915 - val_loss: 0.1927 - val_mse: 0.1927
Epoch 314/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1915 - mse: 0.1915 - val_loss: 0.1927 - val_mse: 0.1927
Epoch 315/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1908 - mse: 0.1908 - val_loss: 0.1919 - val_mse: 0.1919
Epoch 316/320
12/12 [==============================] - 0s 6ms/step - loss: 0.1904 -

In [5]:
encoder_np.shape

(478, 128)

In [6]:
df1 = data[['code', 'label','year','board',
            'industry_code','word_count','sentence_count','readability',
            'pos_count','neg_count','tone']].reset_index(drop=True)
new_df = pd.concat([df1, encoder_np], axis=1)

train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)

feature_names = [i for i in new_df.columns if i not in ['code', 'label']]
len(feature_names)

137

In [7]:
def lgb_model(train, target, test, k, seed, parameters):
    feats = [f for f in train.columns if f not in ['code', 'label']]
    print('Current num of features:', len(feats))

    oof_probs = np.zeros((train.shape[0],))
    output_preds = 0
    offline_score = []
    feature_importance_df = pd.DataFrame()
    parameters = parameters
    seeds = [2]
    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            train_y, test_y = target.iloc[train_index], target.iloc[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]

            dtrain = lgb.Dataset(train_X,
                                 label=train_y)
            dval = lgb.Dataset(test_X,
                               label=test_y)

            lgb_model = lgb.train(
                parameters,
                dtrain,
                num_boost_round=8000,
                valid_sets=[dval],
                callbacks=[early_stopping(100), log_evaluation(100)],
            )

            oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration) / len(
                seeds)

            offline_score.append(lgb_model.best_score['valid_0']['auc'])
            output_preds += lgb_model.predict(test[feats],
                                              num_iteration=lgb_model.best_iteration) / folds.n_splits / len(seeds)
            print(offline_score)
            # feature importance
            fold_importance_df = pd.DataFrame()
            fold_importance_df["feature"] = feats
            fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
            fold_importance_df["fold"] = i + 1
            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(50))

    return output_preds, oof_probs, np.mean(offline_score), feature_importance_df

In [10]:
parameters = {
         'boosting_type': 'gbdt','objective': 'binary',
         'tree_learner':'serial','metric': 'auc',
            'max_depth' : 6,
            'num_leaves' : 15,
            'min_child_weight': 0.01,
            'min_child_samples': 19,
            'num_leaves': 40,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 16,
            'learning_rate': 0.2,
            'silent': True,
            'verbose': -1
    }

print('开始模型训练train')
lgb_preds, lgb_oof, lgb_score, feature_importance_df = lgb_model(
    train=train_df[feature_names],target=train_df['label'],
    test=test_df[feature_names], k=5,seed=42, parameters=parameters)

开始模型训练train
Current num of features: 137
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.693657
Early stopping, best iteration is:
[7]	valid_0's auc: 0.774966
[0.774966261808367]
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.761134


D:\Anaconda3\lib\site-packages\lightgbm\basic.py:1487: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


Early stopping, best iteration is:
[34]	valid_0's auc: 0.813765
[0.774966261808367, 0.8137651821862348]
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.783241
Early stopping, best iteration is:
[10]	valid_0's auc: 0.850416
[0.774966261808367, 0.8137651821862348, 0.850415512465374]
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.836565
Early stopping, best iteration is:
[11]	valid_0's auc: 0.893352
[0.774966261808367, 0.8137651821862348, 0.850415512465374, 0.8933518005540166]
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.774775
Early stopping, best iteration is:
[28]	valid_0's auc: 0.802495
[0.774966261808367, 0.8137651821862348, 0.850415512465374, 0.8933518005540166, 0.8024948024948025]
OOF-MEAN-AUC:0.826999, OOF-STD-AUC:0.041071
featur

In [11]:
val_pred = lgb_oof.copy()
t0 = 0.1
v = 0.02
best_t = t0
best_f1 = 0
for step in range(1000):
    curr_t = t0 + step * v
    val_y = [1 if x >= curr_t else 0 for x in val_pred]
    curr_f1 = f1_score(train_df['label'], val_y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best auc: {}'.format(step, best_t, best_f1))
print('search finish.')
label=[1 if x >= best_t else 0 for x in lgb_preds]
sum(label)
acc = accuracy_score(label, test_df['label'])
# 输出准确率
print('Accuracy:', acc)

step: 0   best threshold: 0.1   best auc: 0.6911764705882354
step: 1   best threshold: 0.12000000000000001   best auc: 0.704119850187266
step: 2   best threshold: 0.14   best auc: 0.7137404580152672
step: 3   best threshold: 0.16   best auc: 0.7181467181467182
step: 4   best threshold: 0.18   best auc: 0.7322834645669292
step: 5   best threshold: 0.2   best auc: 0.738095238095238
step: 6   best threshold: 0.22   best auc: 0.7454909819639278
step: 8   best threshold: 0.26   best auc: 0.7458333333333333
step: 9   best threshold: 0.28   best auc: 0.75
step: 10   best threshold: 0.30000000000000004   best auc: 0.755939524838013
step: 11   best threshold: 0.32   best auc: 0.756637168141593
step: 12   best threshold: 0.33999999999999997   best auc: 0.7606263982102908
step: 13   best threshold: 0.36   best auc: 0.761904761904762
step: 16   best threshold: 0.42000000000000004   best auc: 0.7637231503579951
step: 17   best threshold: 0.44000000000000006   best auc: 0.7658536585365853
search fin